# Factory Methodパターン
*  オブジェクトの作成をサブクラスに委ねることで，利用者側（クライアント／別のシステム）を具体的なクラスのインスタンス化から独立させる
*  オブジェクトの生成とオブジェクトの具体的な処理（実装）を分離することで（単一責任の原則遵守），柔軟にオブジェクトを利用でき再利用性を高める  
*  直感的なイメージ:
>*  製品（オブジェクト）を生成する工場（クラス）のテンプレートがある
>*  具体的な工場（車工場や船工場）はこのテンプレートに基づいて作られる
>*  これらの具体的な工場から製品が生成される
*  具体的には，インターフェースで処理の骨組みを作り，サブクラスを用いてオブジェクトを作成する
*  サブクラスに応じて，作成されるオブジェクトのタイプを変える 


## Factory Methodパターンの構成要素
*  Product:
>*  生成されるオブジェクトの構成要素を定義するインターフェースまたは抽象クラス
>*  具体的な構成要素は, サブクラス（具象クラス）のConcreteProductで定義
*  ConcreteProduct:
>*  Productインターフェースを実装する具体的なクラス（複数作成）
*  Creator:
>*  Productのオブジェクト（インスタンス）を生成する処理を定義したインターフェースまたは抽象クラス
>*  具体的な生成処理（`create`）については, サブクラス（具象クラス）のConcreteCreatorで定義
*  ConcreteCreator:
>*  Creatorクラスを継承し，Creatorを具体化するクラス
>*  ConcreteProductのオブジェクト（インスタンス）を生成する具体的な処理を定義したクラス


> <img src='./fig/FactoryMethod_templete.jpg' width='750'>

## Factory Methodパターンの例 その1
*  以下のコードでは，異なる種類の乗り物オブジェクト（車と船）を作成するためのFactory Methodパターンを実装している
*  `Factory`クラス（インターフェース）はCreatorで，抽象メソッド`create`を持つ
*  `create`メソッドは，乗り物のオーナーの名前を引数として受け取る
*  ConcreteCreatorである`CarFactory`クラスと`ShipFactory`クラスは，`Factory`に対する具象クラスとして`create`メソッドの実装をそれぞれ定義する
*  実装されたメソッドは，ConcreteProductである`Car`クラスのインスタンスと`Ship`クラスのインスタンスをそれぞれ生成する
*  `Car`クラスと`Ship`クラスのスーパークラスが抽象クラスの`Product`で，抽象メソッド`use`を持つ
*  `use`メソッドの実装は，`Car`クラスと`Ship`クラスでそれぞれ定義する
*  この例のように，Factory Methodパターンは，具体的な乗り物オブジェクトの生成を利用者（クライアント／他のシステム）から隠蔽し，動的にオブジェクトを生成している
*  これにより，柔軟かつ拡張可能な設計を実現する
*  Factory Methodパターンは，オブジェクトの作成プロセスが複雑な場合や，利用者側で利用するオブジェクトが属するクラスの詳細を知らなくてもよい場合に特に有効に働く

> <img src='./fig/design_patterns_factorymethod_example.jpg' width='800'>


In [ ]:
from abc import ABC, abstractmethod

# Product
class Product(ABC):

    def __init__(self, owner: str) -> None:
        self._owner = owner

    @abstractmethod
    def use(self) -> None:
        pass

# ConcreteProduct
class Car(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the car.')

# ConcreteProduct
class Ship(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the ship.')

# Creator
class Factory(ABC):

    @abstractmethod
    def create(self, owner: str) -> Product:
        pass

# ConcreteCreator
class CarFactory(Factory):

    def create(self, owner: str) -> Car:
        return Car(owner)

# ConcreteCreator
class ShipFactory(Factory):

    def create(self, owner: str) -> Ship:
        return Ship(owner)

# クライアント
def client_code(factory: Factory, owner: str) -> None:
    product: Product = factory.create(owner)
    product.use()

car_factory: Factory = CarFactory()
ship_factory: Factory = ShipFactory()

client_code(car_factory, 'Alice')
client_code(car_factory, 'Bob')

client_code(ship_factory, 'John')
client_code(ship_factory, 'Paul')

## Factory Methodパターンの例 その2
* ここでは，LSP（リスコフの置換原則）に違反しているコードを，Factory Methodパターンを使ってLSP遵守のコードに改善する例を考える
* LSPについては，[第10回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook10.ipynb)を参照



### LSPに違反している例
* 以下のコードは，音楽プレイヤーをイメージしたコードになっている（下図はそのクラス図）
* このコードにおいて，スーパークラスとなる `MusicPlayer` で「プレーヤーを使って音楽を再生する」という契約を結んでいる
* この契約は，`play`メソッドに対応しており，引数`file`として再生するファイル（`str`）を受け取る形になっている
* しかし，サブクラスは上記契約を破っている ⇒ LSP違反
>* `Mp3Player` は「拡張子が .mp3 であること」を追加前提としている
>* `WavPlayer` はさらに「`driver_loaded` が `True`（WAVドライバ読み込み済み）」 という前提まで要求している
* ドライバが読み込まれてない場合（`driver_loaded`が`False`）は，OSレベルの操作（ファイルや入出力）で失敗したときに起こる例外である OSError を発生させている
* `if`文の条件式で用いている `endswith` は，引数で指定した文字列（接尾辞）で終わっているかどうかを調べるメソッド（[第2回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook02.ipynb)参照）

> <img src='./fig/factorymethod_player1.jpg' width='420'>

**コードに潜在する問題:**
* クライアントは「再生失敗時に行う共通の処理」を前提にできない
* `WavPlayer` がデフォルトで再生不能な状態（`driver_loaded: bool=False`） ⇒ クライアントは，常に音楽が再生できるという期待を持てない


In [ ]:
from abc import ABC, abstractmethod

# スーパークラス
class MusicPlayer(ABC):
    @abstractmethod
    def play(self, file: str) -> None:
        pass

# サブクラス
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise ValueError('mp3ファイルだけ再生できます')
        print(f'[MP3] {file} を再生しました')

# サブクラス
class WavPlayer(MusicPlayer):
    def __init__(self, driver_loaded: bool = False) -> None:
        self.driver_loaded = driver_loaded

    def play(self, file: str) -> None:
        if not self.driver_loaded:
            raise OSError('WAVドライバが読み込まれていません')
        if not file.endswith('.wav'):
            raise ValueError('wavファイルだけ再生できます')
        print(f'[WAV] {file} を再生しました')


# クライアント
def client_code(player: MusicPlayer, file: str):
    player.play(file)

client_code(Mp3Player(), 'song.mp3')
client_code(WavPlayer(), 'song.wav') # 例外発生

### LSPを遵守したコードに改善
* ここで，Factory Methodパターンを使ってLSPを遵守したコードに改善する
* 改善したコードは以下のとおりで，クラス図は下図のように描ける

> <img src="./fig/factorymethod_player2.jpg" width="580">

**Factory Methodパターンの構成要素:**
>* Creator: `MusicApp`クラス
>* ConcreteCreator: `DefaultMusicApp`クラス
>* Product: `MusicPlayer`クラス
>* ConcreteProduct: `Mp3Player`クラスと`WavPlayer`クラス

**Productの生成について:**
*  `create`メソッドは，再生するファイル名を引数として受け取る
*  `create`メソッドは，`play_music`メソッドの中で呼び出される ⇒ ファイル再生時に，ファイルの種類に応じて再生可能なプレイヤーを生成する
*  つまり，Productを Creator 経由で適切に選択・生成する

**失敗時の処理について:**
*  独自の例外「PlaybackError」を定義し，再生失敗時にこの例外を発生させる ⇒ 失敗時の共通の処理パターンが確立できる
*  これにより，クライアントは「失敗時は PlaybackError を捕捉すればよい」という期待で `MusicPlayer` を利用できる
*  また，`WavPlayer` はコンストラクタ内で必ずドライバを読み込んでいるので，`WavPlayer`自身で条件を満たすことができる

In [ ]:
from abc import ABC, abstractmethod

# 独自の例外: 失敗時に発生
class PlaybackError(Exception):
    pass

# Product
class MusicPlayer(ABC):
    @abstractmethod
    def play(self, file: str) -> None:
        pass

# ConcreteProduct
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise PlaybackError('unsupported format')
        print(f'[MP3] {file} を再生しました')

# ConcreteProduct
class WavPlayer(MusicPlayer):
    def __init__(self) -> None:
        print('WAVドライバを読み込みました') # ここで必ずロードして再生条件を満たす
        self._driver_loaded: bool = True

    def play(self, file: str) -> None:
        if not self._driver_loaded:
            raise PlaybackError('driver not loaded')
        if not file.endswith('.wav'):
            raise PlaybackError('unsupported format')
        print(f'[WAV] {file} を再生しました')

# Creator
class MusicApp(ABC):
    @abstractmethod
    def create(self, file: str) -> MusicPlayer:
        pass

    def play_music(self, file: str) -> None:
        player = self.create(file)
        player.play(file)

# ConcreteCreator
class DefaultMusicApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        if file.endswith('.mp3'):
            print('MP3プレイヤーを起動します')
            return Mp3Player()
        if file.endswith('.wav'):
            print('WAVプレイヤーを起動します')
            return WavPlayer()
        raise PlaybackError('未対応の音楽形式です')

# クライアント
def client_code(app: MusicApp, file: str) -> None:
    app.play_music(file)

app: MusicApp = DefaultMusicApp()
client_code(app, 'song.mp3')
client_code(app, 'song.wav')

### コードの拡張
* 上記のコードの拡張を考える
* ConcreteProductの追加: FLAC用プレイヤー `FlacPlayer` を追加する
* ConcreteCreatorの追加: 常に`Mp3Player`だけを生成する `Mp3OnlyApp` を追加する（軽量アプリのようなイメージ）
* 追加・修正した点
>* `FlacPlayer`の定義
>* `FlacPlayer`のインスタンス生成のために `DefaultMusicApp` に`if`文を追加
>* `Mp3OnlyApp`の定義
* 他の部分は変更なし

In [ ]:
from abc import ABC, abstractmethod

# 独自の例外: 失敗時に発生
class PlaybackError(Exception):
    pass

# Product
class MusicPlayer(ABC):
    @abstractmethod
    def play(self, file: str) -> None:
        pass

# ConcreteProduct
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise PlaybackError('unsupported format')
        print(f'[MP3] {file} を再生しました')

# ConcreteProduct
class WavPlayer(MusicPlayer):
    def __init__(self) -> None:
        print('WAVドライバを読み込みました') # ここで必ずロードして再生条件を満たす
        self._driver_loaded: bool = True

    def play(self, file: str) -> None:
        if not self._driver_loaded:
            raise PlaybackError('driver not loaded')
        if not file.endswith('.wav'):
            raise PlaybackError('unsupported format')
        print(f'[WAV] {file} を再生しました')

# ConcreteProduct（新規追加）
class FlacPlayer(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith(".flac"):
            raise PlaybackError('unsupported format')
        print(f"[FLAC] {file} を再生しました")

# Creator
class MusicApp(ABC):
    @abstractmethod
    def create(self, file: str) -> MusicPlayer:
        pass

    def play_music(self, file: str) -> None:
        player = self.create(file)
        player.play(file)

# ConcreteCreator
class DefaultMusicApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        if file.endswith('.mp3'):
            print('MP3プレイヤーを起動します')
            return Mp3Player()
        if file.endswith('.wav'):
            print('WAVプレイヤーを起動します')
            return WavPlayer()
        # FlacPlayerを生成するためのif文追加
        if file.endswith('.flac'):
            print('FLACプレイヤーを起動します')
            return FlacPlayer()
        raise PlaybackError('未対応の音楽形式です')

# ConcreteCreator（新規追加）
class Mp3OnlyApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        return Mp3Player()

# クライアント
def client_code(app: MusicApp, file: str) -> None:
    app.play_music(file)

app1: MusicApp = DefaultMusicApp()
client_code(app1, 'song.mp3')
client_code(app1, 'song.wav')
client_code(app1, 'song.flac') # FLACファイルの再生

app2: MusicApp = Mp3OnlyApp()
client_code(app2, 'song.mp3')
client_code(app2, 'song.wav') # 例外発生


## Factory Methodパターンのメリット・デメリット

| 観点         | メリット                                          | デメリット                                 |
| ---------- | --------------------------------------------- | ------------------------------------- |
| **拡張性**    | 新しい製品を追加するときは，新しいサブクラスを追加するだけで済む（OCP遵守）   | 製品が増えるたびにサブクラスが増えてクラス数が多くなる      |
| **責任分離**   | 「製品を作る部分」と「製品を使う部分」を分けられる（SRP遵守）           | クラスの関係が間接的になるため，初心者には理解が難しい           |
| **柔軟性**    | 実行時に「どの ConcreteCreator を使うか」を切り替えることで，生成物を動的に選択可能 | Creator と Product の両方の階層設計が必要になり，設計が重くなる |


# Singletonパターン
*  Singleton（シングルトン）パターンは，特定クラスのインスタンスが一つだけ存在することを保証するデザインパターン
*  一般にインスタンスを作成すると，それを格納するためのメモリ領域を確保する必要があるので，同じクラスからのインスタンスが何個も必要ない場合には，このパターンを用いて，メモリの無駄遣いを防ぐ
*  Singletonパターンは，システム全体で共有されるリソースや設定を管理する場合などに使用される
>*  例えば，Factory Methodパターンで定義したConcreteCreatorのインスタンスは，誰が使っても同じなので，インスタンスは一つあれば十分である
>*  このような場合に，Singletonパターンが用いられる
*  その他の利用例：データベースの接続，設定情報の管理，ログの記録など

## すべてのクラスのスーパークラス`Object`
*  Pythonでは，自作クラスも既存のクラスもすべて`Object`クラスのサブクラスとして定義されている
*  任意のクラスのスーパークラスを調べたい場合は，そのクラスに定義されている`__bases__`属性（変数）を参照すればよい

**書式:** `クラス名.__base__`

In [ ]:
class MyClass:
    pass

print(MyClass.__base__)

## `__new__`メソッド
*  `__new__`メソッドは特殊メソッドの一つで，クラス自身を引数として，インスタンス生成前に必ず呼び出されるクラスメソッド
*  クラスメソッドについては，[第5回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook05.ipynb)を参照
*  `__new__`メソッドは，`__init__`メソッド（コンストラクタ）よりも先に呼び出される
*  `__new__`メソッドは，クラスのインスタンスを生成して返す役割を持つ
>*  インスタンスを返さない（インスタンスが生成されない）と`__init__`メソッドが実行されない
>*  つまり，`__new__`メソッドでインスタンスを生成してから，`__init__`メソッドで属性の初期化が行われる
*  `__new__`メソッドは，Singletonパターンの実装などでよく使われる ⇒ 具体的には，`Object`クラス（スーパークラス）の`__new__`メソッドをオーバーライドする
*  次のサンプルコードにおける`super().__new__(cls)`は，スーパークラス（つまり`Object`クラス）の`__new__`メソッドを呼び出している
*  `__new__`メソッドの戻り値の型ヒントは「MyClass」となるが，[第6回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook06.ipynb)でも述べているように，クラスの定義が完了する前にそのクラス名を型ヒントとして使用すると例外や警告が発生するため，文字列「'MyClass'」で指定している

In [ ]:
# インスタンスを返すケース
class MyClass:
    def __new__(cls) -> 'MyClass':
        print('__new__の処理')
        return super().__new__(cls)

    def __init__(self) -> None:
        print('__init__の処理')

obj: MyClass = MyClass()
print(type(obj))

In [ ]:
# インスタンスを返さないケース
class MyClass:
    def __new__(cls) -> None:
        print('__new__の処理')

    def __init__(self) -> None:
        print('__init__の処理')

obj: None = MyClass() # インスタンスが生成されない ⇒ Noneとなる
print(type(obj))
print(obj)

## Singletonパターンの基本形
*  ただ一つのインスタンスしか生成できないクラス（`Singleton`クラス）を定義
*  そのクラスのただ一つのインスタンスを保持するためのクラス属性`instance`を定義（クラス属性・インスタンス属性については[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）
*  クラス属性`instance`の初期値は`None`にしておく ⇒ インスタンス化されるまでは何も格納されない
*  クラスメソッドである`__new__`メソッドを，ただ一つのインスタンスを生成するためのメソッドにオーバーライドする（オーバーライドについては[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）
*  `Singleton`クラスを継承したクラスのインスタンスも一つだけしか存在しない

## Singletonパターンの例 その1
*  以下のコードでは2つのインスタンス`a`と`b`を作成しており，それらが違うインスタンスであることを `is`演算子で確かめている
*  `is`演算子については，[第6回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook06.ipynb)を参照

In [ ]:
class DataBase:
    pass

a: DataBase = DataBase()
b: DataBase = DataBase()
print(a is b)

*  上のコードにSingletonパターンを適用すると以下のようになる
*  可視性をプライベートとしたクラス属性`__instance`は，インスタンスが生成されていない場合は`None`で，インスタンス化されると，そのインスタンスが格納される
*  そのため型ヒントは，そのクラス自身`Self`，または`None`を表す `Optional[Self]` となる
*  typyingモジュールの `Self` は「このクラス自身」を表す型ヒントとして使う
*  2回目以降のインスタンス化では，`__new__`メソッドの`if`文が`False`となり，既にあるインスタンスが返される


In [ ]:
from typing import Optional, Self

class Singleton:
    
    __instance: Optional[Self] = None

    def __new__(cls) -> Self: # __new__のオーバーライド
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

a: Singleton = Singleton()
b: Singleton = Singleton()
print(a is b)

## Singletonパターンの例 その2
* 前述した「Factory Methodパターンの例 その1」のコードにSingletonパターンを適用する
* この例において，車の工場（`CarFactory`）や船の工場（`ShipFactory`）は，1つあればいいので，これらのクラスにSingletonパターンを適用する

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional, Self

# Product
class Product(ABC):
    def __init__(self, owner: str) -> None:
        self._owner = owner

    @abstractmethod
    def use(self) -> None:
        pass

# ConcreteProduct
class Car(Product):
    def use(self) -> None:
        print(f'{self._owner} is driving the car.')

# ConcreteProduct
class Ship(Product):
    def use(self) -> None:
        print(f'{self._owner} is driving the ship.')

# Creator
class Factory(ABC):
    @abstractmethod
    def create(self, owner: str) -> Product:
        pass

# Singleton
class Singleton:
    __instance: Optional[Self] = None

    def __new__(cls) -> Self:
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

# ConcreteCreator
class CarFactory(Factory, Singleton):
    def create(self, owner: str) -> Car:
        return Car(owner)

# ConcreteCreator
class ShipFactory(Factory, Singleton):
    def create(self, owner: str) -> Ship:
        return Ship(owner)

# クライアント
def client_code(factory: Factory, owner: str) -> None:
    product: Product = factory.create(owner)
    product.use()

car_factory: Factory = CarFactory()
ship_factory: Factory = ShipFactory()

client_code(car_factory, 'Alice')
client_code(car_factory, 'Bob')

client_code(ship_factory, 'John')
client_code(ship_factory, 'Paul')

# Factoryを2つ生成
car_factory2: Factory = CarFactory()
ship_factory2: Factory = ShipFactory()
print(car_factory is car_factory2)
print(ship_factory is ship_factory2)

## Singletonパターンのメリット・デメリット

| 観点           | メリット                                         | デメリット                                    |
| ------------ | -------------------------------------------- | ---------------------------------------- |
| **管理のしやすさ** | インスタンスが1つだけなので，状態を一元管理できる | どこからでも同じものにアクセスできるため．あちこちで使われた場合には，テストや変更が難しくなる   |
| **アクセスのしやすさ** | 唯一のインスタンスにどこからでもアクセスできる  | 「共通のデータ置き場」のようになり，依存関係が増えて設計が複雑になりやすい      |
| **設計の簡単さ**    | 導入が簡単で，コード量も少ない  | 安易に使うと，あらゆるところから利用される共通の変数なってしまい，設計が崩れやすい   |


# インターフェース分離の原則（ISP: Interface Segregation Principle）

## ISPとは
*  システムを設計する際には，使っていないもの（メソッドなど）への依存を回避すべきだという原則
>*  つまり，クライアント（利用者）が利用しないメソッドへの依存を強制してはならないという原則
*  インタフェースに必要のないメソッドを追加して，継承先が無駄なコードを作成することがないようにすべき
*  インターフェースの持つメソッドは必要最小限のものにすべき ⇒ 小さく特定の役割に特化したインターフェースは，再利用しやすい
*  インターフェースの持つメソッドを増やしすぎると…
>*  継承先のクラスに無駄なコードが増え，コードの複雑化を引き起こす
>*  インターフェースの継承先が増えるため，インターフェースの修正による，継承先クラスの修正量が多くなり保守性が低下する
*  ISPに違反することによる影響:
>*  不要な依存関係が生じることで，クラス間の依存関係が強くなる
>*  インターフェース（抽象クラス）の機能が多くなるので，SRPに違反する可能性が高い
>*  結果的にコードの変更が難しくなる⇒保守性の低下
*  ISPに違反したコードを改善するために，インターフェース（抽象クラス）を分割する ⇒ 多重継承や集約（コンポジション）を使う（多重継承については[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）

## ISPの具体例

### ISPを違反した例
*  以下のコードでは，多機能プリンタに関連する処理（メソッド）を定義するインターフェース`MultiFunctionPrinter`を定義している
*  このインターフェースを使用すると，プリント機能のみの単機能プリンタやスキャナーのような機器でも，このインターフェースの中のすべてのメソッドを実装する必要がある
*  これは非効率的で，ISPに違反している
*  ISPに違反している理由:
>*  `SimplePrinter`や`SimpleScanner`は，`MultiFunctionPrinter`を継承している（依存関係がある）
>*  このとき，`SimplePrinter`には必要のない`scan_document`メソッド等の実装を行う必要がある⇒使ってないものへの依存を強制している
>*  `SimpleScanner`も同様

> <img src='./fig/SOLID_ISP_sample1.jpg' width='450'>

In [ ]:
from abc import ABC, abstractmethod

# 抽象クラス（インターフェース）
class MultiFunctionPrinter(ABC):
    @abstractmethod
    def print_document(self) -> None:
        pass

    @abstractmethod
    def scan_document(self) -> None:
        pass

    @abstractmethod
    def fax_document(self) -> None:
        pass

    @abstractmethod
    def copy_document(self) -> None:
        pass

# 印刷機能のみを持つ
class SimplePrinter(MultiFunctionPrinter):
    def print_document(self) -> None:
        print('ドキュメントを印刷')

    # 使用しない機能も実装しなければならない
    def scan_document(self) -> None:
        print('スキャン機能はありません')

    def fax_document(self) -> None:
        print('Fax機能はありません')

    def copy_document(self) -> None:
        print('コピー機能はありません')

# スキャナ機能のみを持つ
class SimpleScanner(MultiFunctionPrinter):
    def print_document(self) -> None:
        print('印刷機能はありません')

    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

    def fax_document(self) -> None:
        print('Fax機能はありません')

    def copy_document(self) -> None:
        print('コピー機能はありません')

# クライアント
printer: MultiFunctionPrinter = SimplePrinter()
printer.print_document()

scanner: MultiFunctionPrinter = SimpleScanner()
scanner.scan_document()

### 多重継承を使ってISPを遵守したコードに改善
*  ISPに従うために，インターフェースを分割する
*  具体的には，それぞれの機能（印刷，スキャン，ファックス，コピー）に対して独立したインターフェースを定義し，必要な機能を持つインターフェースだけを多重継承する
>*  `SimplePrinter`クラス: 印刷機能のみを持つ
>*  `SimpleScanner`クラス: スキャン機能のみを持つ
>*  `PrinterScanner`クラス: 印刷機能とスキャナ機能を持つ
*  これにより，単機能の機器や特定の機能のみを提供する機器に対して，必要なインターフェースのみを実装するだけでよくなり，余計な実装をしなくてすむ
*  また，多重継承を使うと（後述の集約を使ったコードより）シンプルで理解しやすい構造にすることができる

> <img src='./fig/SOLID_ISP_sample2.jpg' width='550'>

In [ ]:
from abc import ABC, abstractmethod

class Printer(ABC):
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    @abstractmethod
    def copy_document(self) -> None:
        pass

# 印刷機能のみを持つ
class SimplePrinter(Printer):
    def print_document(self) -> None:
        print('ドキュメントを印刷')

# スキャナ機能のみを持つ
class SimpleScanner(Scanner):
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

# 印刷機能とスキャナ機能を持つ
class PrinterScanner(Printer, Scanner): #多重継承
    def print_document(self) -> None:
        print('ドキュメントを印刷')

    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

# クライアント
printer: SimplePrinter = SimplePrinter()
printer.print_document()
print('-------------------------------')
scanner: SimpleScanner = SimpleScanner()
scanner.scan_document()
print('-------------------------------')
printer_scanner: PrinterScanner = PrinterScanner()
printer_scanner.print_document()
printer_scanner.scan_document()

### 集約（コンポジション）を使ってISPを遵守したコードに改善
* 付録参照

## ISPを遵守することによる効果
*  小さなインターフェースを設計することで，クライアントは必要なメソッドのみを利用できるようになる
>*  一つのインターフェースに多くの機能（メソッド）が含まれている場合，クライアントがそのインターフェースの一部だけを必要とする場合でも，全体を実装する必要があり，不必要なメソッドの変更が他に影響を与える可能性がある
*  必要なメソッドだけに依存するため，システムを拡張する際に他のクラスやインターフェースに影響を与えにくくなる
*  各インターフェースが単一の責任に絞られる（ISPはインターフェースに対するSRPともいえる）
*  インターフェースが小さくなるため，変更が必要な場合でも，影響範囲が限定される
>*  大きなインターフェースを変更する場合，そのインターフェースを実装しているすべてのクラスに影響を与える可能性がある
>*  一方，小さなインターフェースを使用していれば，その影響は限定的になる
*  クライアントは必要なインターフェースにのみ依存するため，システム全体の依存関係が簡素化される
*  ISPを遵守することで設計の手間は増えるかもしれないが，基本的にはメリットのほうが大きい

## ISPとフールプルーフ
*  ISPにより，利用者が操作できる範囲を制限し，誤操作を防ぐことができる
*  例えば，爆破ボタン付きのスマートフォンを開発し，利用者に「このボタンは押さないでください」といっても，間違って押してしまう利用者がいるかもしれないので，そのようなボタンは最初から作らないほうがよい
*  ISPにより，インターフェースをシンプルにすることができる ⇒ その結果，利用者の誤解を防ぎ，安全に操作できる設計を実現できる

# 依存性逆転の原則（DIP: Dependency Inversion Principle）


## DIPとは
*  コードの依存関係が抽象だけを参照している
>*  抽象とは，抽象クラスやインターフェースと解釈すればよい
*  上位のモジュール（クラス）は下位のモジュール（クラス）に直接依存すべきではなく，両者は抽象に依存すべきであるという原則
*  抽象に依存する⇒具象に依存しない
>*  変化しやすい具象クラスを参照・継承しない
>*  具象メソッドをオーバーライドしない

## 上位モジュールと下位モジュールの例
*  ネットショップにおける注文処理を上位モジュールとする
*  上位モジュールは注文処理を行うという目的を持つ
*  注文処理モジュールの下位モジュールは，上位モジュールの目的を達成するための手段に対応
*  注文処理を実現するためには，例えば，ユーザが存在するかどうか（ユーザの有効性）のチェック，注文された商品の在庫があるかのチェック，支払い処理，在庫情報の更新などの処理が必要となる
*  この例の場合の下位モジュール：
>*  ユーザの有効性をチェックするモジュール
>*  商品の在庫をチェックするモジュール
>*  支払い処理を行うモジュール
>*  在庫情報の更新処理を行うモジュール
*  この例からもイメージできるように，上位モジュールの目的を実現するために，複数の小さな処理（手段）に分割したものが下位モジュールとなる
*  したがって，この考え方では，上位モジュールが下位モジュールに依存することになる
*  つまり，手段の変更によって目的を変えなければならないといった問題が発生する
*  この依存関係を解消する（依存の方向性を逆転する）必要があるという原則がDIPとなる

## DIPを遵守するための方法
*  インターフェースや抽象クラスを使用する
>*  上位モジュールと下位モジュールを，共通のインターフェースまたは抽象クラスに依存させる
>*  これにより，上位モジュールは，下位モジュールの実装を直接参照しないため，依存性が逆転する
*  Dependency Injection（DI: 依存性注入）パターンを使用する（付録参照）
>*  DIパターンは，デザインパターンの一つ（GoFのデザインパターンではない）
>*  GoFのデザインパターンではないので，ソフトウェア設計における手法・原則として位置づけられることもある
>*  必要な下位モジュールを直接作成せずに外部から提供する
>*  これにより，上位モジュールが具体的な下位モジュールに依存しなくなる

## DIPの具体例

### DIPを違反している例
*  以下のコードでは，`EmailService`クラスと`Notification`クラスを定義している  
  
*  `EmailService`クラス:
>*  具体的な通知方法（メール）を定義するクラス（下位モジュール）
>*  任意のメッセージ`msg`をメールで送信する`send_email`メソッドが定義されている
* `Notification`クラス:
>*  個々の通知を表すためのクラス（上位モジュール）
>*  多数の利用者に対して様々な通知を送るシステムをイメージしている
>*  通知サービスオブジェクト`service`として，`EmailService`クラスのオブジェクトが用いられており，任意のメッセージ`msg`を`send_email`メソッドを呼び出して通知する`send`メソッドが定義されている
>*  つまり，`Notification`クラスは，メッセージを通知するためのクラスであるが，通知方法が具体的な方法である`EmailService`クラス（具象クラス）の`send_email`メソッド（具象メソッド）に依存している ⇒ DIPに違反

> <img src='./fig/SOLID_DIP_sample1.jpg' width='550'>  

*  `Notification`クラスと`EmailService`クラスの間には依存関係（メソッドを呼び出す側と呼び出される側という一時的な関係）とコンポジションの関連がある

In [ ]:
class EmailService:
    
    def send_email(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class Notification:
    
    def __init__(self) -> None:
        self.__service: EmailService = EmailService()

    def send(self, msg: str) -> None:
        self.__service.send_email(msg)

notification: Notification = Notification()
notification.send('Hello!')

### DIPを違反していることによる影響
*  上のコードで，通知方法をメールから別の方法に変更する場合には，新たな通知方法（SMSやプッシュ通知など）を定義したクラスを追加するだけでなく，`Notification`クラスを変更する必要もある（下のクラス図とコード参照）
*  これは非効率的であり，DIPに違反している（具体的なクラスやメソッドを参照している）

> <img src='./fig/SOLID_DIP_sample2.jpg' width='550'>  

In [ ]:
class EmailService:
    
    def send_email(self, msg: str) -> None:
        print(f'メール送信: {msg}')

# 新たな通知方法
class ShortMessageService:
    def send_sms(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

# 通知方法を変更するには，この中も変更する必要がある
class Notification:
    def __init__(self) -> None:
        self.__service: ShortMessageService = ShortMessageService() # 変更箇所

    def send(self, msg: str) -> None:
        self.__service.send_sms(msg)

notification: Notification = Notification()
notification.send('Hello!')

### DIPを遵守したコードに改良
*  メッセージ通知のための抽象的な`MessageService`インターフェース（上位モジュール）を定義
*  `MessageService`で抽象メソッド`send_message`を定義
*  `MessageService`インターフェースを実装する具体的なクラス（`MessageService`を継承したクラス）である`EmailService`クラス（下位モジュール）を定義
*  `EmailService`で`send_message`メソッドの実装を定義
*  `Notification`クラス（上位モジュール）の`service`属性は，型ヒントを用いて抽象（`MessageService`）に依存させた形にする
*  `send`メソッドも抽象メソッドである`send_message`に依存させる  

> <img src='./fig/SOLID_DIP_sample3.jpg' width='520'>  

In [ ]:
from abc import ABC, abstractmethod

class MessageService(ABC):
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class Notification:
    def __init__(self, service: MessageService) -> None: # 引数を抽象（MessageService）に依存させる
        self.__service = service

    def send(self, msg: str) -> None:
        self.__service.send_message(msg) # 抽象メソッドに依存（実装はEmailserviceで定義）

notification: Notification = Notification(EmailService())
notification.send('Hello!')

### DIPを遵守していることによる効果
*  インターフェースを用いることにより，`Notification`クラスを，抽象的な`MessageService`インターフェースに依存させた形にできた
*  これにより，具体的な通知方法（具象メソッド）に直接依存することがなくなり，拡張性と柔軟性が高い設計が可能となる  
*  例えば，新たな通知方法としてSMSを追加する場合は，具体的な通知方法（`ShortMessageService`クラス）を追加するだけで済む

> <img src='./fig/SOLID_DIP_sample4.jpg' width='620'>  

In [ ]:
from abc import ABC, abstractmethod

class MessageService(ABC):
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

#他の部分の変更なしでクラスを追加するだけでよい
class ShortMessageService(MessageService):
    def send_message(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

class Notification:
    def __init__(self, service: MessageService) -> None:
        self.__service = service

    def send(self, msg: str) -> None:
        self.__service.send_message(msg)

notification_email = Notification(EmailService())
notification_email.send('こんにちは！')
notification_sms = Notification(ShortMessageService())
notification_sms.send('Hello!')

## DIPを遵守することによる効果
*  下位モジュールの具体的な実装を変更しても，上位モジュールや他の部分に影響を与えない
*  上位モジュールを異なる環境や状況で再利用しやすくなる
*  下位モジュールの実装が未完成な場合でも，テスト用の簡易的なモジュール使ったテストが実行可能
*  新しい下位モジュールを追加しても，上位モジュールを変更する必要がない ⇒ OCPにも貢献
*  つまり，DIPを遵守することで，変更と拡張に強く，テストしやすいコードになる

# Observerパターン
* Observer（オブザーバ）パターンとは，オブジェクト間の依存関係を管理するためのパターン
* あるオブジェクト（Subject）の状態を監視し，状態が変更されたときに，その変更内容を受け取り，依存する他のオブジェクト（Observer）へ自動的に通知し，更新させるしくみを提供する
* 利用例: SNSで投稿した際に，フォロワーへ投稿したことのメッセージを通知する

## Observerパターンの構成要素・基本形
* **Subject**: Observerに通知する必要がある情報を保持し，Observerの登録，削除，及び通知の機能（メソッド）を提供するインターフェースまたは抽象クラスのこと
* **Observer**: Subjectの状態変更に対する更新機能を持つインターフェースまたは抽象クラスで，Subjectの状態変更時に呼び出されるメソッドを実装する
* **ConcreteSubject**: 具体的なSubjectのことで，実際のデータや状態を保持し，状態が変化した際にObserverへ通知する
* **ConcreteObserver**: Subjectの状態変更に応じて具体的な処理を実行する

**基本形のクラス図**:

> <img src='./fig/Observer_templete.jpg' width='450'>  

* Subjectのメソッド:
>* `register` : Observerを登録する
>* `remove` : 登録されているObserverを削除する
>* `notify` : Observerに通知する
* Observerのメソッド:
>* `update` : 通知を表示する
* 基本的には，ObserverがSubjectの構成要素となり，Subjectの中でupdateが呼び出される形となる


## Observerパターンの例

### 天気情報アプリ
* 単純な例として，Subject（アプリをイメージすればよい）が情報を通知したときに，Observer（アプリ利用者の画面をイメージすればよい）にその通知を表示するシステムを考える
* 対応するクラス図は下図のように描ける

> <img src='./fig/Observer_sample1.jpg' width='580'>  

* `Subject`:
>* Subjectの役割を担うインターフェース
>* 3つの抽象メソッド`register`, `remove`, `notify`を定義
>* それぞれ，Observer（アプリ利用者）の登録，削除，通知の処理に対応
* `Observer`:
>* Observerの役割を担うインターフェース
>* 抽象メソッド`update`を定義
>* `update`で，アプリ利用者の画面に通知を表示する
* `WeatherData`:
>* インスタンス属性として，Observerオブジェクトを要素としたリストを保持するための`__observers: list[Observer]`と通知する情報を保持するための`__message: str`を定義
>* Observer（アプリ利用者）の登録，削除，通知の処理である `register`, `remove`, `notify` を実装
>* さらに固有のメソッドとして，天気情報を通知するための`set_measurements`メソッドを定義
* `Display`:
>* アプリ利用者が持つスマホ画面だと考えればよい
>* インスタンス属性として，アプリ利用者の名前を保持するための`__name: str`を定義
>* アプリ利用者の画面に通知（天気情報）を表示する `update` を実装
>* `update`では，アプリ利用者の名前（`__name`）と引数`message: str`で受け取った通知を表示する
* この例において，`Subject`インターフェースの実装`WeatherData`が ConcreteSubjectの役割を担う「天気情報アプリ」に対応し，`Observer`インターフェースの実装`Display`が ConcreteObserverの役割を担う「アプリ利用者の画面」に対応する
* 天気情報が表示されるまでの処理の流れ:
>* まず，`set_measurements`メソッドによって天気情報が通知されると，その中で `notify`メソッドが呼び出される
>* `notify`メソッドの中で，登録されているObserver固有の`update`メソッドを呼び出すことで通知が表示される
* Observerパターンを使用することで，情報を通知した場合に，すべての登録されたObserverにその情報を効率的に伝えることができる
* また，新しいObserver / Subject の追加や既存のObserver / Subject の削除が容易になり，システムの柔軟性と拡張性が向上する（具体例は付録参照）

In [ ]:
from abc import ABC, abstractmethod

# Observer
class Observer(ABC):
    @abstractmethod
    def update(self, message: str) -> None:
        pass

# Subject
class Subject(ABC):
    @abstractmethod
    def register(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def remove(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

# ConcreteSubject（天気情報アプリ）
class WeatherData(Subject):
    def __init__(self) -> None:
        self.__observers: list[Observer] = []
        self.__message: str = ''

    def register(self, observer: Observer) -> None:
        self.__observers.append(observer)

    def remove(self, observer: Observer) -> None:
        self.__observers.remove(observer)
    
    def notify(self) -> None:
        for obs in self.__observers:
            obs.update(f'[天気] {self.__message}')

    def set_measurements(self, message: str) -> None:
        self.__message = message
        self.notify()


# ConcreteObserver（アプリ利用者）
class Display(Observer):
    def __init__(self, name: str) -> None:
        self.__name = name

    def update(self, message: str) -> None:
        print(f'{self.__name}の画面に通知: {message}')


# クライアント
weather: Subject = WeatherData()

taro: Observer = Display('太郎')
hanako: Observer = Display('花子')

weather.register(taro)
weather.register(hanako)

weather.set_measurements('気温25℃ 湿度60%')

weather.remove(taro)

weather.set_measurements('気温28℃ 湿度70%')


## Observerパターンのメリット・デメリット

| 観点         | メリット                               | デメリット                |
| ---------- | ---------------------------------- | -------------------- |
| **通知の仕組み** | 1対多の通知を自動化できる                      | Observerが多いと通知コストが大きくなる |
| **依存関係**   | 主体（Subject）とObserverが「直接強く結びつかない」ので変更に強い | 通知の流れが見えにくく，デバッグが難しい |
| **拡張性**    | 新しいObserverを追加しても主体（Subject）を変更せず拡張できる（OCP遵守） | 通知順序やタイミングを制御しにくい    |



# 実習
以下の「修正前コード」は，「問題点」で指摘しているように依存性逆転の原則（DIP）に違反している．このコードを「コード修正要件・補足説明」に従って修正し，DIP遵守のコードに修正しなさい．ただし，コードには型ヒントをつけ，既に入力されているコードは変更・削除しないこと．

---
**修正前コード：**
```Python
class Lamp:
    def on(self) -> None:
        print('Lamp: ON')

    def off(self) -> None:
        print('Lamp: OFF')

class LedStrip:
    def on(self) -> None:
        print('LED Strip: ON')
    
    def off(self) -> None:
        print('LED Strip: OFF')

class Switch:
    def __init__(self) -> None:
        self.__lamp: Lamp = Lamp()   # ← ここが直接依存（DIP違反）

    def press(self, on: bool) -> None:
        if on:
            self.__lamp.on()
        else:
            self.__lamp.off()
```
---
**問題点:**
* このコードでは，`Switch`（上位モジュール）が具体的な `Lamp`（下位モジュール）に依存している（抽象に依存していない）ため，DIPを違反している
* 例えば，スイッチ（`Switch`）の切り替え先を電球（`Lamp`）からLEDテープ（`LedStrip`）に変更したいときには，`Switch`の内容を書き換えなければならない
---
**コード修正要件・補足説明:**
*  以下のクラス図と対応させる
*  `Lamp`と`LedStrip`における `on`と`off`は修正前と同じ
*  `Switch`の `__init__`と`press`は，インスタンス属性`__light`に対応させた形に修正する
---
**クラス図:**

<img src='./fig/SOLID_DIP_exercise.jpg' width='450'>

---
**期待される実行結果:**
```
Lamp: ON
Lamp: OFF
LED Strip: ON
LED Strip: OFF
```

In [ ]:
# ここにコードを記述（必要に応じて改行する）

# --------------------------
# 動作確認（クライアント側）
# --------------------------
sw1: Switch = Switch(Lamp())
sw1.press(True)
sw1.press(False)

sw2: Switch = Switch(LedStrip())
sw2.press(True)
sw2.press(False)


# 参考資料
*  Robert C.Martin (著), 角征典 (翻訳), 高木正弘 (翻訳), [Clean Architecture 達人に学ぶソフトウェアの構造と設計](https://www.kadokawa.co.jp/product/301806000678/), KADOKAWA, 2018.
*  ひらまつしょうたろう, [Python で身につける オブジェクト指向【SOLID原則+デザインパターンで、オブジェクト指向設計 の基礎を習得！】](https://www.udemy.com/course/python-solid-design-pattern/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2024/6.
*  Mark Summerfield (著), 斎藤康毅 (訳), [実践 Python 3](https://www.oreilly.co.jp/books/9784873117393/), オライリージャパン, 2015.
<!-- *  Bill Lubanovic (著), 鈴木駿 (監訳), 長尾高弘 (訳), [入門 Python 3 第2版](https://www.oreilly.co.jp/books/9784873119328/), オライリージャパン, 2021. -->
<!-- *  Guido van Rossum (著), 鴨澤眞夫 (翻訳), [Pythonチュートリアル 第4版](https://www.oreilly.co.jp/books/9784873119359/), オライリージャパン, 2021. -->



# 付録

## 集約（コンポジション）を使ってISPを遵守したコードに改善する例
* ここでは，2段階のステップを踏んで改善していく

### 第1段階
*  前述したコード例と同様に，それぞれの機能（印刷，スキャン，ファックス，コピー）に対して独立したインターフェースを定義する ⇒ `Printer`, `Scanner`, `Fax`, `Copier`
*  各インターフェースの実装を定義するクラスをそれぞれ定義する ⇒ `BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`
*  この例にはないが，各インターフェースの実装は複数定義することもできる
>*  例えば，より高速な印刷やスキャンを行う「`HighSpeed​​Printer`」や「`HighSpeed​​Scanner`」などを定義することもできる
*  上記のクラスのオブジェクトの一部を部分として持つクラスを定義する ⇒ `SimplePrinter`, `SimpleScanner`, `PrinterScanner`
*  このクラスをインスタンス化する際に，インスタンス属性となるインターフェースの実装を定義するクラスのオブジェクト（`BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`）を引数として指定する
*  このとき指定する引数は抽象に依存した形になっている
*  また，`BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`は，StrategyパターンにおけるStrategyと解釈することもできる
*  上述した`HighSpeed​​Printer`や`HighSpeed​​Scanner`などといった別のStrategyを定義することで，Contentの役割を持つ`SimplePrinter`, `SimpleScanner`, `PrinterScanner`がStrategyを選択できるようになる

> <img src='./fig/SOLID_ISP_sample3.jpg' width='1000'>

In [ ]:
from abc import ABC, abstractmethod

class Printer(ABC):
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    @abstractmethod
    def copy_document(self) -> None:
        pass

class BasicPrinter(Printer):
    def print_document(self) -> None:
        print('ドキュメントを印刷')

class BasicScanner(Scanner):
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

class BasicFax(Fax):
    def fax_document(self) -> None:
        print('ドキュメントをFaxで送信')

class BasicCopier(Copier):
    def copy_document(self) -> None:
        print('ドキュメントをコピー')
    
# 印刷機能のみを持つ
class SimplePrinter:
    def __init__(self, printer: Printer) -> None:
        self.printer = printer
    
    def execute_print(self) -> None:
        self.printer.print_document()

# スキャナ機能のみを持つ
class SimpleScanner:
    def __init__(self, scanner: Scanner) -> None:
        self.scanner = scanner
    
    def execute_scan(self) -> None:
        self.scanner.scan_document()

# 印刷機能とスキャナ機能を持つ
class PrinterScanner:
    def __init__(self, printer: Printer, scanner: Scanner) -> None:
        self.printer = printer
        self.scanner = scanner
    
    def execute_print(self) -> None:
        self.printer.print_document()

    def execute_scan(self) -> None:
        self.scanner.scan_document()

# クライアント
printer: SimplePrinter = SimplePrinter(BasicPrinter())
printer.execute_print()
print('-------------------------------')
scanner: SimpleScanner = SimpleScanner(BasicScanner())
scanner.execute_scan()
print('-------------------------------')
printer_scanner: PrinterScanner = PrinterScanner(BasicPrinter(), BasicScanner())
printer_scanner.execute_print()
printer_scanner.execute_scan()

### 第2段階
*  上のコードにおける，`SimplePrinter`, `SimpleScanner`, `PrinterScanner`を一つクラスとしてまとめることを考える
*  異なる機能を持つデバイスを柔軟に構成するための`MultiFunctionDevice`クラスを新たに定義
*  `MultiFunctionDevice`クラスをインスタンス化する際に渡す引数によって，様々なバリエーションのデバイスが表現できる
>*  印刷機能のみを持つデバイス: `MultiFunctionDevice(printer = BasicPrinter())`
>*  スキャン機能のみを持つデバイス: `MultiFunctionDevice(scanner = BasicScanner())`
>*  印刷機能とスキャナ機能を持つデバイス: `MultiFunctionDevice(printer = BasicPrinter(), scanner = BasicScanner())`
*  引数はすべてデフォルト値を`None`としているので，必要な機能のみをキーワード指定で指定すればよい（キーワード指定については[第3回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook03.ipynb)を参照）
*  `MultiFunctionDevice`クラスの各メソッド（`print_document`や`scan_document`など）は，該当するオブジェクトが設定されていない（属性が`None`である）場合に，その機能が使えないといったメッセージを表示する
*  このように設計することで，オブジェクトが未設定のために生じる不具合を回避できる
*  ただし，この設計（コード）では，クライアントからみると必要のないメソッドを公開している（正常に呼び出せる）ので，例外を発生させて例外として処理をしたほうが望ましいともいえる

> <img src='./fig/SOLID_ISP_sample4.jpg' width='900'>

In [ ]:
from typing import Optional
from abc import ABC, abstractmethod

class Printer(ABC):
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    @abstractmethod
    def copy_document(self) -> None:
        pass

class BasicPrinter(Printer):
    def print_document(self) -> None:
        print('ドキュメントを印刷')

class BasicScanner(Scanner):
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

class BasicFax(Fax):
    def fax_document(self) -> None:
        print('ドキュメントをFaxで送信')

class BasicCopier(Copier):
    def copy_document(self) -> None:
        print('ドキュメントをコピー')

class MultiFunctionDevice:
    def __init__(self, printer: Optional[Printer] = None, scanner: Optional[Scanner] = None,
                 fax: Optional[Fax] = None, copier: Optional[Copier] = None) -> None:
        self.printer = printer
        self.scanner = scanner
        self.fax = fax
        self.copier = copier

    def execute_print(self) -> None:
        if self.printer:
            self.printer.print_document()
        else:
            print('印刷機能はありません')

    def execute_scan(self) -> None:
        if self.scanner:
            self.scanner.scan_document()
        else:
            print('スキャン機能はありません')

    def execute_fax(self) -> None:
        if self.fax:
            self.fax.fax_document()
        else:
            print('Fax機能はありません')

    def execute_copy(self) -> None:
        if self.copier:
            self.copier.copy_document()
        else:
            print('コピー機能はありません')

# 印刷機能のみを持つデバイス
simple_printer: MultiFunctionDevice = MultiFunctionDevice(printer = BasicPrinter())
simple_printer.execute_print()
simple_printer.execute_scan()
print('-------------------------------')
# スキャナ機能のみを持つデバイス
simple_scanner: MultiFunctionDevice = MultiFunctionDevice(scanner = BasicScanner())
simple_scanner.execute_print()
simple_scanner.execute_scan()
print('-------------------------------')
# 印刷機能とスキャナ機能を持つデバイス
printer_scanner: MultiFunctionDevice = MultiFunctionDevice(printer = BasicPrinter(), scanner = BasicScanner())
printer_scanner.execute_print()
printer_scanner.execute_scan()

## Dependency Injection（DI: 依存性注入）パターン
*  クラスが直接依存関係（下位モジュール）を生成するのではなく，下位モジュールを外部から注入（提供）するしくみ
*  DIパターンにおける注入方法には，コンストラクタ注入やセッター注入などがある
*  一般的には，コンストラクタ注入が使われる

###  コンストラクタ注入の例
*  前述した`EmailService`クラスと`Notification`クラスを実装したコード例（下図）がコンストラクタ注入の例になっている
*  `Notification`クラスの`sendメソッド`は，具体的なメッセージ送信手段に依存するが，その依存関係（下位モジュール）を外部から注入している
*  具体的には，`MessageService`クラスに対する具象クラス（`EmailService`クラスや`ShortMessageService`クラス）のオブジェクトをコンストラクタに渡している

> <img src='./fig/SOLID_DIP_sample4.jpg' width='650'>  

In [ ]:
# 前述のコード例と同じ
from abc import ABC, abstractmethod

class MessageService(ABC):
    
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class ShortMessageService(MessageService):
    
    def send_message(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

class Notification:
    
    def __init__(self, service: MessageService) -> None:
        self.__service = service

    def send(self, msg: str) -> None:
        self.__service.send_message(msg)

notification_email: Notification = Notification(EmailService()) # 依存関係注入
notification_email.send('こんにちは！')
notification_sms = Notification(ShortMessageService()) # 依存関係注入
notification_sms.send('Hello!')

### セッター注入の例
*  上のコードと同じ機能を持つが，依存関係の注入方法が異なる
*  具体的には，`Notification`クラスの定義を変更している
>*  `__init__`メソッド（コンストラクタ）で依存関係を注入しない ⇒ 初期値として`None`を設定
>*  `service`属性に対するゲッターとセッターを定義
>*  ゲッターを使って依存関係を注入する ⇒ インスタンス生成後，`service`属性に下位モジュールのオブジェクトを設定する
>*  `send`メソッドにおいて，依存関係が注入されてなかった場合（初期値`None`から変更されてない場合）は，例外オブジェクトを発生させる

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

class MessageService(ABC):
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class ShortMessageService(MessageService):
    def send_message(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

class Notification:
    def __init__(self) -> None:
        self.__service: Optional[MessageService] = None # 依存関係を注入しない

    @property # ゲッター（__serviceの取得）
    def service(self) -> Optional[MessageService]:
        return self.__service

    @service.setter # セッター（__serviceの設定）
    def service(self, service: MessageService):
        self.__service = service

    def send(self, msg: str) -> None:
        if self.__service is None: # self.__serviceがNoneかどうかを判定
            raise ValueError('MessageService is not set.') # 例外オブジェクト発生
        self.__service.send_message(msg)

notification: Notification = Notification()
notification.service = EmailService() # 依存関係注入
notification.send('こんにちは！')
notification.service = ShortMessageService() # 依存関係注入
notification.send('Hello!')

## Observerパターンの例
* ここで，前述したアプリと利用者のコード例の拡張を考える

### 新しいSubjectの追加
* 天気情報アプリに加えて，株価を通知する株アプリ`StockData`を追加することを考える
* このとき，他のクラスに影響することなく `StockData` を追加できる ⇒ 高い拡張性を実現
* `WeatherData`の定義と異なる点:
>* インスタンス属性: 天気情報を保持する`__message`ではなく，株価を保持する`__price`を定義
>* 固有のメソッド: 天気情報を通知する`set_measurements`ではなく，株価を通知する`set_price`を定義

> <img src='./fig/Observer_sample2.jpg' width='800'>  

In [ ]:
from abc import ABC, abstractmethod

# Observer
class Observer(ABC):
    @abstractmethod
    def update(self, message: str) -> None:
        pass

# Subject
class Subject(ABC):
    @abstractmethod
    def register(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def remove(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

# ConcreteSubject（天気情報アプリ）
class WeatherData(Subject):
    def __init__(self) -> None:
        self.__observers: list[Observer] = []
        self.__message: str = ''

    def register(self, observer: Observer) -> None:
        self.__observers.append(observer)

    def remove(self, observer: Observer) -> None:
        self.__observers.remove(observer)
    
    def notify(self) -> None:
        for obs in self.__observers:
            obs.update(f'[天気] {self.__message}')

    def set_measurements(self, message: str) -> None:
        self.__message = message
        self.notify()

# 新規追加: ConcreteSubject（株アプリ）
class StockData(Subject):
    def __init__(self) -> None:
        self.__observers: list[Observer] = []
        self.__price: float = 0.0

    def register(self, observer: Observer) -> None:
        self.__observers.append(observer)

    def remove(self, observer: Observer) -> None:
        self.__observers.remove(observer)
    
    def notify(self) -> None:
        for obs in self.__observers:
            obs.update(f'[株価] {self.__price}円')

    def set_price(self, price: float) -> None:
        self.__price = price
        self.notify()

# ConcreteObserver（アプリ利用者）
class Display(Observer):
    def __init__(self, name: str) -> None:
        self.__name = name

    def update(self, message: str) -> None:
        print(f'{self.__name}の画面に通知: {message}')

# クライアント
weather: Subject = WeatherData()
stock: Subject = StockData()

taro: Observer = Display('太郎')
hanako: Observer = Display('花子')

weather.register(taro)
weather.register(hanako)
weather.set_measurements('気温25℃ 湿度60%')
weather.remove(taro)
weather.set_measurements('気温28℃ 湿度70%')

stock.register(taro)
stock.set_price(1250.5)
stock.register(hanako)
stock.set_price(1251.2)

### 新しいObserverの追加
* 現状の`Display`クラスは「通知をそのまま出力する」だけ
* ここで，通知をログに保存する`Logger`という新しいObserverを追加することを考える
>* ログを保持しておくためのインスタンス属性`__logs: list[str]`を定義
>* ログを`__logs`に追加して，メッセージを表示する`update`メソッドを実装
>* ログを順番に表示する`show_logs`メソッドを定義
* このとき，他のクラスに影響することなく `Logger` を追加できる ⇒ 高い拡張性を実現

> <img src='./fig/Observer_sample3.jpg' width='950'>  

In [ ]:
from abc import ABC, abstractmethod

# Observer
class Observer(ABC):
    @abstractmethod
    def update(self, message: str) -> None:
        pass

# Subject
class Subject(ABC):
    @abstractmethod
    def register(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def remove(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

# ConcreteSubject（天気情報アプリ）
class WeatherData(Subject):
    def __init__(self) -> None:
        self.__observers: list[Observer] = []
        self.__message: str = ''

    def register(self, observer: Observer) -> None:
        self.__observers.append(observer)

    def remove(self, observer: Observer) -> None:
        self.__observers.remove(observer)
    
    def notify(self) -> None:
        for obs in self.__observers:
            obs.update(f'[天気] {self.__message}')

    def set_measurements(self, message: str) -> None:
        self.__message = message
        self.notify()

# ConcreteSubject（株アプリ）
class StockData(Subject):
    def __init__(self) -> None:
        self.__observers: list[Observer] = []
        self.__price: float = 0.0

    def register(self, observer: Observer) -> None:
        self.__observers.append(observer)

    def remove(self, observer: Observer) -> None:
        self.__observers.remove(observer)
    
    def notify(self) -> None:
        for obs in self.__observers:
            obs.update(f'[株価] {self.__price}円')

    def set_price(self, price: float) -> None:
        self.__price = price
        self.notify()

# ConcreteObserver（アプリ利用者）
class Display(Observer):
    def __init__(self, name: str) -> None:
        self.__name = name

    def update(self, message: str) -> None:
        print(f'{self.__name}の画面に通知: {message}')

# 新規追加: ConcreteObserver（通知を保存するアプリ）
class Logger(Observer):
    def __init__(self) -> None:
        self.__logs: list[str] = []

    def update(self, message: str) -> None:
        self.__logs.append(message)
        print(f'Logger: メッセージを記録しました -> {message}')

    def show_logs(self) -> None:
        print('=== 保存されたログ ===')
        for log in self.__logs:
            print(log)
        print('====== ここまで ======')

# クライアント
weather: Subject = WeatherData()
stock: Subject = StockData()

taro: Observer = Display('太郎')
hanako: Observer = Display('花子')
logger1: Observer = Logger() # 新しいObserver

weather.register(taro)
weather.register(hanako)
weather.register(logger1) # Loggerも登録
weather.set_measurements('気温25℃ 湿度60%')
weather.remove(taro)
weather.set_measurements('気温28℃ 湿度70%')
logger1.show_logs() # ログの表示

logger2: Observer = Logger() # 新しいObserver
stock.register(taro)
stock.register(hanako)
stock.register(logger2) # Loggerも登録
stock.set_price(1250.5)
stock.set_price(1251.2)
logger2.show_logs() # ログの表示